In [1]:
from nba_api.stats.endpoints import SynergyPlayTypes, LeagueDashPtStats, CommonPlayerInfo, CommonTeamYears , CommonTeamRoster, CommonAllPlayers
from supabase import create_client, Client
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os
import time

In [2]:
load_dotenv("../.env.local")

SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")

In [3]:
supabase = create_client(SUPABASE_URL, SUPABASE_KEY)

In [4]:
# Current NBA season
season = "2025-26"

print(f"Pulling players for season {season}")
response = CommonAllPlayers(
    is_only_current_season=1,   # Only active players
    league_id="00",
    season=season
)

df = response.get_data_frames()[0]

# Filter: only currently rostered players
df = df[df["ROSTERSTATUS"] == 1]

print(f"Collected {len(df)} active players for {season}")
df

Pulling players for season 2025-26
Collected 524 active players for 2025-26


,PERSON_ID,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FIRST_LAST,ROSTERSTATUS,FROM_YEAR,TO_YEAR,PLAYERCODE,PLAYER_SLUG,TEAM_ID,TEAM_CITY,TEAM_NAME,TEAM_ABBREVIATION,TEAM_SLUG,TEAM_CODE,GAMES_PLAYED_FLAG,OTHERLEAGUE_EXPERIENCE_CH
0,1630173,"Achiuwa, Precious",Precious Achiuwa,1,2020,2025,precious_achiuwa,precious_achiuwa,1610612758,Sacramento,Kings,SAC,kings,kings,Y,00
1,203500,"Adams, Steven",Steven Adams,1,2013,2025,steven_adams,steven_adams,1610612745,Houston,Rockets,HOU,rockets,rockets,Y,00
2,1628389,"Adebayo, Bam",Bam Adebayo,1,2017,2025,bam_adebayo,bam_adebayo,1610612748,Miami,Heat,MIA,heat,heat,Y,00
3,1630534,"Agbaji, Ochai",Ochai Agbaji,1,2022,2025,ochai_agbaji,ochai_agbaji,1610612761,Toronto,Raptors,TOR,raptors,raptors,Y,00
4,1630583,"Aldama, Santi",Santi Aldama,1,2021,2025,santi_aldama,santi_aldama,1610612763,Memphis,Grizzlies,MEM,grizzlies,grizzlies,Y,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541,1642911,"Zikarsky, Rocco",Rocco Zikarsky,1,2025,2025,rocco_zikarsky,rocco_zikarsky,1610612750,Minnesota,Timberwolves,MIN,timberwolves,timberwolves,Y,11
542,1627826,"Zubac, Ivica",Ivica Zubac,1,2016,2025,ivica_zubac,ivica_zubac,1610612746,LA,Clippers,LAC,clippers,clippers,Y,01
543,1641783,"da Silva, Tristan",Tristan da Silva,1,2024,2025,tristan_da_silva,tristan_da_silva,1610612753,Orlando,Magic,ORL,magic,magic,Y,00
544,1642365,"Đurišić, Nikola",Nikola Đurišić,1,2025,2025,nikola_durisic,nikola_đurišić,1610612737,Atlanta,Hawks,ATL,hawks,hawks,Y,11


In [ ]:
# Create a list of unique player IDs
player_ids = df["PERSON_ID"].tolist()

uploaded_players = pd.read_csv("players_rows.csv")

unuploaded_player_ids = df.loc[
    ~df["PERSON_ID"].isin(uploaded_players["player_id"]),
    "PERSON_ID"
].tolist()

print(unuploaded_player_ids)



[1630828, 1642846, 1642964, 1642926, 1642866, 1642886, 1641780, 1642868, 1642928, 1630619, 1642383, 1642363, 1631451, 1642907, 1643018, 1642885, 1630621, 1642856, 1642845, 1642400, 1642855, 1642847, 1642843, 1642853, 1642066, 1642884, 1642864, 1642844, 1642935, 1642857, 1642883, 1643052, 1642848, 1642938, 1642880, 1642357, 1641750, 1642939, 1642851, 1642404, 1642502, 1631126, 1642863, 1642918, 1642942, 1643024, 1642862, 1642867, 1642948, 1642854, 1642949, 1642950, 1642877, 1642869, 1643007, 1642962, 1642878, 1642852, 1642875, 1642954, 1642859, 1642860, 1642920, 1642879, 1642917, 1642914, 1642850, 1642876, 1642910, 1642849, 1642364, 1642873, 1642874, 1642905, 1642959, 1642911, 1642365]


In [11]:

detailed_players = []
failed_ids = []
consecutive_failures = 0

for pid in unuploaded_player_ids:
    success = False  # track if we succeed within 3 tries

    for attempt in range(3):
        try:
            info = CommonPlayerInfo(league_id_nullable="00", player_id=pid).get_data_frames()[0]

            row = {
                "player_id": str(info.loc[0, "PERSON_ID"]),
                "player_name": info.loc[0, "DISPLAY_FIRST_LAST"],
                "position": info.loc[0, "POSITION"],
                "height": info.loc[0, "HEIGHT"],
                "weight": info.loc[0, "WEIGHT"],
                "school": info.loc[0, "SCHOOL"],
                "birthdate": info.loc[0, "BIRTHDATE"],
                "draft_year": info.loc[0, "DRAFT_YEAR"],
                "draft_round": info.loc[0, "DRAFT_ROUND"],
                "draft_number": info.loc[0, "DRAFT_NUMBER"]
            }

            detailed_players.append(row)
            success = True
            print(pid)
            consecutive_failures = 0  # reset streak on success
            break  # exit retry loop on success

        except Exception as e:
            print(f"Attempt {attempt + 1} failed for {pid}: {e}")
            time.sleep(30 + attempt)

    if not success:
        failed_ids.append(pid)
        consecutive_failures += 1

    if consecutive_failures >= 2:
        print("\n Two players in a row failed completely. Exiting loop early.")
        break

    time.sleep(5)  # base delay between each player fetch

# Convert to DataFrame
player_info_df = pd.DataFrame(detailed_players)

print(player_info_df)

1630828
1642846
1642964
1642926
1642866
1642886
1641780
1642868
1642928
1630619
1642383
1642363
1631451
1642907
1643018
1642885
1630621
1642856
1642845
1642400
1642855
1642847
1642843
1642853
1642066
1642884
1642864
1642844
1642935
1642857
1642883
1643052
1642848
1642938
1642880
1642357
1641750
1642939
1642851
1642404
1642502
1631126
1642863
1642918
1642942
1643024
1642862
1642867
1642948
1642854
1642949
1642950
1642877
1642869
1643007
1642962
1642878
1642852
1642875
1642954
1642859
1642860
1642920
1642879
1642917
1642914
1642850
1642876
1642910
1642849
1642364
1642873
1642874
1642905
1642959
1642911
1642365
   player_id         player_name position height weight  \
0    1630828  Alex Antetokounmpo  Forward    6-8    214   
1    1642846          Ace Bailey  Forward    6-9    200   
2    1642964    Brooks Barnhizer    Guard    6-5    230   
3    1642926         Tamar Bates    Guard    6-4    195   
4    1642866       Joan Beringer  Forward   6-11    230   
..       ...                 .

In [13]:
pd.DataFrame(
    unuploaded_player_ids,
    columns=["PERSON_ID"]
).to_csv("player_ids.csv", index=False)
player_info_df.to_csv("player_info_full.csv", index=False)

In [16]:
player_info_df.to_csv("player_info_full.csv", index=False)

In [19]:
player_info_df["player_id"] = pd.to_numeric(player_info_df["player_id"], errors="coerce")
player_info_df = player_info_df.dropna(subset=["player_id"])
player_info_df["player_id"] = player_info_df["player_id"].astype(int)

player_info_df = player_info_df.drop_duplicates(subset=["player_id"])

# helps avoid  input syntax
player_info_df = player_info_df.replace({np.nan: None})

records = player_info_df.to_dict(orient="records")

chunk_size = 50
for i in range(0, len(records), chunk_size):
    chunk = records[i:i + chunk_size]
    try:
        supabase.table("players").insert(chunk).execute()
        print(f"Inserted chunk {i // chunk_size + 1}")
        time.sleep(0.3)  # 1s is fine too; 0.3 usually works
    except Exception as e:
        print(f"Error inserting chunk {i // chunk_size + 1}: {e}")

Inserted chunk 1
Inserted chunk 2


In [11]:
nba_teams = CommonTeamYears(league_id='00').get_data_frames()[0]


active_nba_teams = nba_teams[nba_teams["MAX_YEAR"] == '2025']


team_ids = active_nba_teams["TEAM_ID"].tolist()


team_rosters = []
consecutive_failures = 0


    
for team_id in team_ids:

    success = False
    
    for attempt in range(3):
        try:
            info = CommonTeamRoster(team_id=team_id, season=season).get_data_frames()[0]
            
            for _, row in info.iterrows():
                player = {
                    "season": season,
                    "team_id": team_id,
                    "player_id": str(row["PLAYER_ID"]),
                    "player_name": row["PLAYER"],
                    "position": row["POSITION"],
                    "age": row["AGE"],
                    "experience": row["EXP"]
                }
                team_rosters.append(player)
            
            success = True
            consecutive_failures = 0  # reset failure streak
            print(team_id)
            break  # success, break retry loop

        except Exception as e:
            print(f"Attempt {attempt + 1} failed for team {team_id} in {season}: {e}")
            time.sleep(30 + attempt)

    if not success:
        consecutive_failures += 1
        if consecutive_failures >= 2:
            print("Two consecutive team failures — exiting loop early.")
            break
    
    time.sleep(5) 



# Convert to DataFrame
team_roster_df = pd.DataFrame(team_rosters)
print(f"Finished with {len(team_roster_df)} player-team-season records.")
print(team_roster_df)

1610612737
1610612738
1610612739
1610612740
1610612741
1610612742
1610612743
1610612744
1610612745
1610612746
1610612747
1610612748
1610612749
1610612750
1610612751
1610612752
1610612753
1610612754
1610612755
1610612756
1610612757
1610612758
1610612759
1610612760
1610612761
1610612762
1610612763
1610612764
1610612765
1610612766
Finished with 523 player-team-season records.
      season     team_id player_id     player_name position   age experience
0    2025-26  1610612737   1630552   Jalen Johnson        F  24.0          4
1    2025-26  1610612737   1630811  Keaton Wallace        G  26.0          1
2    2025-26  1610612737    203468     CJ McCollum        G  34.0         12
3    2025-26  1610612737   1628379    Luke Kennard        G  29.0          8
4    2025-26  1610612737   1630700   Dyson Daniels        G  22.0          3
..       ...         ...       ...             ...      ...   ...        ...
518  2025-26  1610612766   1630544        Tre Mann        G  25.0          4
519  202

In [12]:
team_roster_df["player_id"] = pd.to_numeric(team_roster_df["player_id"], errors='coerce')
team_roster_df["team_id"] = pd.to_numeric(team_roster_df["team_id"], errors='coerce')
team_roster_df["season"] = team_roster_df["season"].astype(str)
team_roster_df = team_roster_df.drop_duplicates(subset=["season", "team_id", "player_id"])
print(team_roster_df)

      season     team_id  player_id     player_name position   age experience
0    2025-26  1610612737    1630552   Jalen Johnson        F  24.0          4
1    2025-26  1610612737    1630811  Keaton Wallace        G  26.0          1
2    2025-26  1610612737     203468     CJ McCollum        G  34.0         12
3    2025-26  1610612737    1628379    Luke Kennard        G  29.0          8
4    2025-26  1610612737    1630700   Dyson Daniels        G  22.0          3
..       ...         ...        ...             ...      ...   ...        ...
518  2025-26  1610612766    1630544        Tre Mann        G  25.0          4
519  2025-26  1610612766    1641706  Brandon Miller        F  23.0          2
520  2025-26  1610612766    1642354      KJ Simpson        G  23.0          1
521  2025-26  1610612766    1642275  Tidjane Salaün        F  20.0          1
522  2025-26  1610612766    1642862   Liam McNeeley        F  20.0          R

[523 rows x 7 columns]


In [13]:
team_roster_df["experience"] = team_roster_df["experience"].replace("R", 0).astype(int)
team_roster_df = team_roster_df.drop_duplicates()
print(team_roster_df)

      season     team_id  player_id     player_name position   age  experience
0    2025-26  1610612737    1630552   Jalen Johnson        F  24.0           4
1    2025-26  1610612737    1630811  Keaton Wallace        G  26.0           1
2    2025-26  1610612737     203468     CJ McCollum        G  34.0          12
3    2025-26  1610612737    1628379    Luke Kennard        G  29.0           8
4    2025-26  1610612737    1630700   Dyson Daniels        G  22.0           3
..       ...         ...        ...             ...      ...   ...         ...
518  2025-26  1610612766    1630544        Tre Mann        G  25.0           4
519  2025-26  1610612766    1641706  Brandon Miller        F  23.0           2
520  2025-26  1610612766    1642354      KJ Simpson        G  23.0           1
521  2025-26  1610612766    1642275  Tidjane Salaün        F  20.0           1
522  2025-26  1610612766    1642862   Liam McNeeley        F  20.0           0

[523 rows x 7 columns]


In [14]:
team_roster_df.to_csv("team_info_full26.csv", index=False)

In [15]:
# Convert to record dicts
team_records = team_roster_df.to_dict(orient="records")

# Upload in chunks
chunk_size = 1000
for i in range(0, len(team_records), chunk_size):
    chunk = team_records[i:i + chunk_size]
    try:
        supabase.table("teams_players").insert(chunk).execute()
        print(f"Inserted chunk {i // chunk_size + 1}")
        time.sleep(1)
    except Exception as e:
        print(f"Error inserting chunk {i // chunk_size + 1}: {e}")


Inserted chunk 1


SYNERGY STATS

In [4]:
# Define the seasons and play types
season = "2025-26"

play_types = [
    "Cut", "Handoff", "Isolation", "Misc", "OffScreen",
    "Postup", "PRBallHandler", "PRRollman", "OffRebound", "Spotup", "Transition"
]

groupings = ["offensive", "defensive"]  # Add both

all_synergy_data = []

# Main data pull loop

for grouping in groupings:
    for play_type in play_types:
        for attempt in range(3):
            try:
                synergy = SynergyPlayTypes(
                    league_id="00",
                    per_mode_simple="PerGame",
                    player_or_team_abbreviation="P",
                    season_type_all_star="Regular Season",
                    season=season,
                    play_type_nullable=play_type,
                    type_grouping_nullable=grouping
                )
                
                df = synergy.get_data_frames()[0]
                df["SEASON"] = season
                df["PLAY_TYPE"] = play_type
                df["TYPE_GROUPING"] = grouping
                all_synergy_data.append(df)
                print(f" {grouping} - {play_type} - {season}")
                break  # success

            except Exception as e:
                print(f"Attempt {attempt + 1} failed: {grouping} | {play_type} | {season} — {e}")
                time.sleep(30 + attempt)

        time.sleep(5)

 offensive - Cut - 2025-26
 offensive - Handoff - 2025-26
 offensive - Isolation - 2025-26
 offensive - Misc - 2025-26
 offensive - OffScreen - 2025-26
 offensive - Postup - 2025-26
 offensive - PRBallHandler - 2025-26
 offensive - PRRollman - 2025-26
 offensive - OffRebound - 2025-26
 offensive - Spotup - 2025-26
 offensive - Transition - 2025-26
 defensive - Cut - 2025-26
 defensive - Handoff - 2025-26
 defensive - Isolation - 2025-26
 defensive - Misc - 2025-26
 defensive - OffScreen - 2025-26
 defensive - Postup - 2025-26
 defensive - PRBallHandler - 2025-26
 defensive - PRRollman - 2025-26
 defensive - OffRebound - 2025-26
 defensive - Spotup - 2025-26
 defensive - Transition - 2025-26


In [5]:
synergy_df = pd.concat(all_synergy_data, ignore_index=True)

synergy_selected_columns = ['SEASON_ID', 'PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'PLAY_TYPE', 'TYPE_GROUPING', 'PERCENTILE', 'POSS_PCT', 'PPP', 'FG_PCT', 'FT_POSS_PCT', 'TOV_POSS_PCT', 'SF_POSS_PCT', 'PLUSONE_POSS_PCT', 'SCORE_POSS_PCT', 'EFG_PCT', 'POSS', 'PTS', 'FGM', 'FGA', 'FGMX', 'SEASON']
synergy_filtered_df = synergy_df[synergy_selected_columns]


print(synergy_filtered_df)

     SEASON_ID PLAYER_ID     PLAYER_NAME     TEAM_ID PLAY_TYPE TYPE_GROUPING  \
0        22025   1641708   Amen Thompson  1610612745       Cut     offensive   
1        22025   1628386   Jarrett Allen  1610612739       Cut     offensive   
2        22025   1629028   Deandre Ayton  1610612747       Cut     offensive   
3        22025   1631105     Jalen Duren  1610612765       Cut     offensive   
4        22025   1629674   Neemias Queta  1610612738       Cut     offensive   
...        ...       ...             ...         ...       ...           ...   
4858     22025   1642843    Cooper Flagg  1610612742    Spotup     defensive   
4859     22025   1630700   Dyson Daniels  1610612737    Spotup     defensive   
4860     22025   1628401   Derrick White  1610612738    Spotup     defensive   
4861     22025   1642273  Kyshawn George  1610612764    Spotup     defensive   
4862     22025   1641824   Matas Buzelis  1610612741    Spotup     defensive   

      PERCENTILE  POSS_PCT    PPP  FG_P

C:\Users\npowers\AppData\Local\Temp\ipykernel_37936\1838134032.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  synergy_df = pd.concat(all_synergy_data, ignore_index=True)


In [16]:
synergy_merged_df = synergy_filtered_df.merge(
    team_roster_df[["player_id", "season", "position", "age", "experience"]],
    left_on=["PLAYER_ID", "SEASON"],
    right_on=["player_id", "season"],
    how="left"
)

synergy_merged_df = synergy_merged_df.dropna(subset=["player_id"])
synergy_merged_df = synergy_merged_df.drop_duplicates()

synergy_merged_df = synergy_merged_df.drop(columns=["player_id","season"], errors="ignore")

print(synergy_merged_df.columns)

Index(['SEASON_ID', 'PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'PLAY_TYPE',
       'TYPE_GROUPING', 'PERCENTILE', 'POSS_PCT', 'PPP', 'FG_PCT',
       'FT_POSS_PCT', 'TOV_POSS_PCT', 'SF_POSS_PCT', 'PLUSONE_POSS_PCT',
       'SCORE_POSS_PCT', 'EFG_PCT', 'POSS', 'PTS', 'FGM', 'FGA', 'FGMX',
       'SEASON', 'position', 'age', 'experience'],
      dtype='object')


In [17]:
synergy_merged_df["PPP_pct_by_age"] = synergy_merged_df.groupby("age")["PPP"].rank(pct=True)
synergy_merged_df["PPP_pct_by_experience"] = synergy_merged_df.groupby("experience")["PPP"].rank(pct=True)

In [18]:
# Define position groups
guard_positions = ["G", "G-F"]
wing_positions = ["F", "F-G"]
forward_positions = ["C", "C-F", "F-C"]

# Create a new column for position group
def get_position_group(pos):
    if pos in guard_positions:
        return "Guard"
    elif pos in wing_positions:
        return "Wing"
    elif pos in forward_positions:
        return "Forward"
    else:
        return "Other"

synergy_merged_df["position_group"] = synergy_merged_df["position"].apply(get_position_group)

# Calculate PPP percentiles by position group
synergy_merged_df["PPP_pct_by_position_group"] = synergy_merged_df.groupby("position_group")["PPP"].rank(pct=True)

print(synergy_merged_df)


     SEASON_ID PLAYER_ID     PLAYER_NAME     TEAM_ID PLAY_TYPE TYPE_GROUPING  \
0        22025   1641708   Amen Thompson  1610612745       Cut     offensive   
1        22025   1628386   Jarrett Allen  1610612739       Cut     offensive   
2        22025   1629028   Deandre Ayton  1610612747       Cut     offensive   
3        22025   1631105     Jalen Duren  1610612765       Cut     offensive   
4        22025   1629674   Neemias Queta  1610612738       Cut     offensive   
...        ...       ...             ...         ...       ...           ...   
4858     22025   1642843    Cooper Flagg  1610612742    Spotup     defensive   
4859     22025   1630700   Dyson Daniels  1610612737    Spotup     defensive   
4860     22025   1628401   Derrick White  1610612738    Spotup     defensive   
4861     22025   1642273  Kyshawn George  1610612764    Spotup     defensive   
4862     22025   1641824   Matas Buzelis  1610612741    Spotup     defensive   

      PERCENTILE  POSS_PCT    PPP  FG_P

In [19]:
for col in ["PPP_pct_by_age", "PPP_pct_by_experience", "PPP_pct_by_position_group"]:
    synergy_merged_df.loc[synergy_merged_df["TYPE_GROUPING"] == "defensive", col] = (
        1 - synergy_merged_df.loc[synergy_merged_df["TYPE_GROUPING"] == "defensive", col]
    )

print(synergy_merged_df)

     SEASON_ID PLAYER_ID     PLAYER_NAME     TEAM_ID PLAY_TYPE TYPE_GROUPING  \
0        22025   1641708   Amen Thompson  1610612745       Cut     offensive   
1        22025   1628386   Jarrett Allen  1610612739       Cut     offensive   
2        22025   1629028   Deandre Ayton  1610612747       Cut     offensive   
3        22025   1631105     Jalen Duren  1610612765       Cut     offensive   
4        22025   1629674   Neemias Queta  1610612738       Cut     offensive   
...        ...       ...             ...         ...       ...           ...   
4858     22025   1642843    Cooper Flagg  1610612742    Spotup     defensive   
4859     22025   1630700   Dyson Daniels  1610612737    Spotup     defensive   
4860     22025   1628401   Derrick White  1610612738    Spotup     defensive   
4861     22025   1642273  Kyshawn George  1610612764    Spotup     defensive   
4862     22025   1641824   Matas Buzelis  1610612741    Spotup     defensive   

      PERCENTILE  POSS_PCT    PPP  FG_P

PLAYTYPE STATS

In [20]:
season = "2025-26"

pt_types = [
    "SpeedDistance", "Rebounding", "CatchShoot", "PullUpShot", "Drives",
    "Passing", "ElbowTouch", "PostTouch", "PaintTouch", "Efficiency"
]

pt_dfs = {pt: [] for pt in pt_types}  # dict to collect dataframes for each pt type


for pt in pt_types:
    try:
        df = LeagueDashPtStats(
            season=season,
            pt_measure_type=pt,
            player_or_team="Player",
            per_mode_simple="PerGame",
            season_type_all_star="Regular Season"
        ).get_data_frames()[0]

        df["season"] = season
        suffix_cols = [col for col in df.columns if col not in ["PLAYER_ID", "TEAM_ID", "season"]]
        df = df.rename(columns={col: f"{col}_{pt}" for col in suffix_cols})

        pt_dfs[pt].append(df)
        print(f"Pulled: {season} | {pt}")
        time.sleep(1)
    except Exception as e:
        print(f"FAILED: {season} | {pt} --> {e}")
        time.sleep(5)

# Combine all dataframes by merging on PLAYER_ID, TEAM_ID, and season
from functools import reduce
merged_pt_df = reduce(
    lambda left, right: pd.merge(left, right, on=["PLAYER_ID", "TEAM_ID", "season"], how="outer"),
    [pd.concat(pt_dfs[pt], ignore_index=True) for pt in pt_types]
)

# Optional preview
print(merged_pt_df.shape)
print(merged_pt_df.columns.tolist())
print(merged_pt_df)

Pulled: 2025-26 | SpeedDistance
Pulled: 2025-26 | Rebounding
Pulled: 2025-26 | CatchShoot
Pulled: 2025-26 | PullUpShot
Pulled: 2025-26 | Drives
Pulled: 2025-26 | Passing
Pulled: 2025-26 | ElbowTouch
Pulled: 2025-26 | PostTouch
Pulled: 2025-26 | PaintTouch
Pulled: 2025-26 | Efficiency
(520, 209)
['PLAYER_ID', 'PLAYER_NAME_SpeedDistance', 'TEAM_ID', 'TEAM_ABBREVIATION_SpeedDistance', 'GP_SpeedDistance', 'W_SpeedDistance', 'L_SpeedDistance', 'MIN_SpeedDistance', 'MIN1_SpeedDistance', 'DIST_FEET_SpeedDistance', 'DIST_MILES_SpeedDistance', 'DIST_MILES_OFF_SpeedDistance', 'DIST_MILES_DEF_SpeedDistance', 'AVG_SPEED_SpeedDistance', 'AVG_SPEED_OFF_SpeedDistance', 'AVG_SPEED_DEF_SpeedDistance', 'season', 'PLAYER_NAME_Rebounding', 'TEAM_ABBREVIATION_Rebounding', 'GP_Rebounding', 'W_Rebounding', 'L_Rebounding', 'MIN_Rebounding', 'OREB_Rebounding', 'OREB_CONTEST_Rebounding', 'OREB_UNCONTEST_Rebounding', 'OREB_CONTEST_PCT_Rebounding', 'OREB_CHANCES_Rebounding', 'OREB_CHANCE_PCT_Rebounding', 'OREB_CH

In [21]:
print(synergy_df.columns.tolist())
print(merged_pt_df.columns.tolist())


['SEASON_ID', 'PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'PLAY_TYPE', 'TYPE_GROUPING', 'PERCENTILE', 'GP', 'POSS_PCT', 'PPP', 'FG_PCT', 'FT_POSS_PCT', 'TOV_POSS_PCT', 'SF_POSS_PCT', 'PLUSONE_POSS_PCT', 'SCORE_POSS_PCT', 'EFG_PCT', 'POSS', 'PTS', 'FGM', 'FGA', 'FGMX', 'SEASON']
['PLAYER_ID', 'PLAYER_NAME_SpeedDistance', 'TEAM_ID', 'TEAM_ABBREVIATION_SpeedDistance', 'GP_SpeedDistance', 'W_SpeedDistance', 'L_SpeedDistance', 'MIN_SpeedDistance', 'MIN1_SpeedDistance', 'DIST_FEET_SpeedDistance', 'DIST_MILES_SpeedDistance', 'DIST_MILES_OFF_SpeedDistance', 'DIST_MILES_DEF_SpeedDistance', 'AVG_SPEED_SpeedDistance', 'AVG_SPEED_OFF_SpeedDistance', 'AVG_SPEED_DEF_SpeedDistance', 'season', 'PLAYER_NAME_Rebounding', 'TEAM_ABBREVIATION_Rebounding', 'GP_Rebounding', 'W_Rebounding', 'L_Rebounding', 'MIN_Rebounding', 'OREB_Rebounding', 'OREB_CONTEST_Rebounding', 'OREB_UNCONTEST_Rebounding', 'OREB_CONTEST_PCT_Rebounding', 'OREB_CHANCES_Rebounding', 'OREB_CHANCE_PCT_Reboundin

In [22]:
playtype_selected_columns = ['PLAYER_ID','season', 'PLAYER_NAME_SpeedDistance', 'TEAM_ID',
                             
                            'DIST_MILES_OFF_SpeedDistance','DIST_MILES_DEF_SpeedDistance', 'AVG_SPEED_SpeedDistance', 
                            'AVG_SPEED_OFF_SpeedDistance', 'AVG_SPEED_DEF_SpeedDistance', # Speed and Distance Group

                            'OREB_Rebounding', 'OREB_CONTEST_Rebounding', 'OREB_CONTEST_PCT_Rebounding',
                            'OREB_CHANCES_Rebounding', 'OREB_CHANCE_PCT_Rebounding', 
                            'OREB_UNCONTEST_Rebounding', 'AVG_OREB_DIST_Rebounding', # Offensive Rebounding Group
                            
                            'DREB_Rebounding', 'DREB_CONTEST_Rebounding', 'DREB_CONTEST_PCT_Rebounding',
                            'DREB_CHANCES_Rebounding', 'DREB_CHANCE_PCT_Rebounding',
                            'DREB_UNCONTEST_Rebounding', 'AVG_DREB_DIST_Rebounding', # Defensive Rebounding Group

                            'CATCH_SHOOT_FGM_CatchShoot','CATCH_SHOOT_FGA_CatchShoot', 'CATCH_SHOOT_FG_PCT_CatchShoot', 
                            'CATCH_SHOOT_FG3M_CatchShoot', 'CATCH_SHOOT_FG3A_CatchShoot', 'CATCH_SHOOT_FG3_PCT_CatchShoot',
                            'CATCH_SHOOT_PTS_CatchShoot', 'CATCH_SHOOT_EFG_PCT_CatchShoot', # Catch and Shoot Group


                            'PULL_UP_FGM_PullUpShot', 'PULL_UP_FGA_PullUpShot', 'PULL_UP_FG_PCT_PullUpShot', 
                            'PULL_UP_FG3M_PullUpShot', 'PULL_UP_FG3A_PullUpShot', 'PULL_UP_FG3_PCT_PullUpShot', # Pull Up Shot Group
                            'PULL_UP_PTS_PullUpShot','PULL_UP_EFG_PCT_PullUpShot', 


                            'DRIVES_Drives', 'DRIVE_PTS_Drives', 'DRIVE_PTS_PCT_Drives',
                            'DRIVE_FGM_Drives', 'DRIVE_FGA_Drives', 'DRIVE_FG_PCT_Drives',
                            'DRIVE_FTA_Drives',
                            'DRIVE_PASSES_PCT_Drives', 'DRIVE_AST_PCT_Drives',
                            'DRIVE_TOV_PCT_Drives', 'DRIVE_PF_PCT_Drives', # Drives Group


                            'PASSES_MADE_Passing', 'PASSES_RECEIVED_Passing', 
                            'AST_Passing', 'SECONDARY_AST_Passing', 'POTENTIAL_AST_Passing', 
                            'FT_AST_Passing',
                            'AST_POINTS_CREATED_Passing', 'AST_ADJ_Passing', 'AST_TO_PASS_PCT_Passing', 
                            # Passing Group


                            'ELBOW_TOUCHES_ElbowTouch', 'ELBOW_TOUCH_PTS_ElbowTouch', 'ELBOW_TOUCH_PTS_PCT_ElbowTouch',
                            'ELBOW_TOUCH_FGM_ElbowTouch', 'ELBOW_TOUCH_FGA_ElbowTouch', 'ELBOW_TOUCH_FG_PCT_ElbowTouch', 
                            'ELBOW_TOUCH_FTA_ElbowTouch', 
                            'ELBOW_TOUCH_AST_PCT_ElbowTouch', 'ELBOW_TOUCH_PASSES_PCT_ElbowTouch',
                            'ELBOW_TOUCH_TOV_PCT_ElbowTouch', 
                            # Elbow Touch Group
                            

                            'POST_TOUCHES_PostTouch', 'POST_TOUCH_PTS_PostTouch', 'POST_TOUCH_PTS_PCT_PostTouch',
                            'POST_TOUCH_FGM_PostTouch', 'POST_TOUCH_FGA_PostTouch', 'POST_TOUCH_FG_PCT_PostTouch', 
                            'POST_TOUCH_FTA_PostTouch',
                            'POST_TOUCH_AST_PCT_PostTouch','POST_TOUCH_PASSES_PCT_PostTouch',
                            'POST_TOUCH_TOV_PCT_PostTouch', 
                            # Post Touch Group

                            'PAINT_TOUCHES_PaintTouch','PAINT_TOUCH_PTS_PaintTouch', 'PAINT_TOUCH_PTS_PCT_PaintTouch',  
                            'PAINT_TOUCH_FGM_PaintTouch', 'PAINT_TOUCH_FGA_PaintTouch', 'PAINT_TOUCH_FG_PCT_PaintTouch', 
                            'PAINT_TOUCH_FTA_PaintTouch', 
                            'PAINT_TOUCH_AST_PCT_PaintTouch','PAINT_TOUCH_PASSES_PCT_PaintTouch', 
                            'PAINT_TOUCH_TOV_PCT_PaintTouch'] # Paint Touch Group


playtype_values_df = merged_pt_df[playtype_selected_columns]

In [23]:
playtype_values_df = playtype_values_df.rename(columns={"PLAYER_NAME_SpeedDistance": "PLAYER_NAME"})

# Define columns to keep as grouping keys (non-numeric)
group_keys = ['PLAYER_ID', 'season', 'PLAYER_NAME', 'TEAM_ID']

# Identify numeric columns to convert to percentiles
numeric_cols = [col for col in playtype_values_df.columns if col not in group_keys]

# Create percentile dataframe
playtype_percentile_df = playtype_values_df[group_keys].copy()

# Calculate percentiles by season
for col in numeric_cols:
    playtype_percentile_df[col + "_pct"] = playtype_values_df.groupby("season")[col].rank(pct=True)

# Invert percentiles for turnover-related columns
for col in numeric_cols:
    if "TOV" in col.upper():
        pct_col = col + "_pct"
        playtype_percentile_df[pct_col] = 1 - playtype_percentile_df[pct_col]


# Preview
print(playtype_percentile_df)


     PLAYER_ID   season      PLAYER_NAME     TEAM_ID  \
0      1630639  2025-26      A.J. Lawson  1610612761   
1      1631260  2025-26         AJ Green  1610612749   
2      1642358  2025-26       AJ Johnson  1610612764   
3       203932  2025-26     Aaron Gordon  1610612743   
4      1628988  2025-26    Aaron Holiday  1610612745   
..         ...      ...              ...         ...   
515    1641744  2025-26        Zach Edey  1610612763   
516     203897  2025-26      Zach LaVine  1610612758   
517    1630192  2025-26       Zeke Nnaji  1610612743   
518    1630533  2025-26  Ziaire Williams  1610612751   
519    1629627  2025-26  Zion Williamson  1610612740   

     DIST_MILES_OFF_SpeedDistance_pct  DIST_MILES_DEF_SpeedDistance_pct  \
0                            0.213462                          0.247115   
1                            0.869231                          0.820192   
2                            0.213462                          0.156731   
3                          

In [24]:
playtype_merged_df = playtype_values_df.merge(
    team_roster_df[["player_id", "season", "position", "age", "experience"]],
    left_on=["PLAYER_ID", "season"],
    right_on=["player_id", "season"],
    how="left"
)

playtype_merged_df = playtype_merged_df.dropna(subset=["player_id"])
playtype_merged_df = playtype_merged_df.drop_duplicates()

playtype_merged_df = playtype_merged_df.drop(columns=["player_id"], errors="ignore")

print(playtype_merged_df)

     PLAYER_ID   season      PLAYER_NAME     TEAM_ID  \
0      1630639  2025-26      A.J. Lawson  1610612761   
1      1631260  2025-26         AJ Green  1610612749   
2      1642358  2025-26       AJ Johnson  1610612764   
3       203932  2025-26     Aaron Gordon  1610612743   
4      1628988  2025-26    Aaron Holiday  1610612745   
..         ...      ...              ...         ...   
515    1641744  2025-26        Zach Edey  1610612763   
516     203897  2025-26      Zach LaVine  1610612758   
517    1630192  2025-26       Zeke Nnaji  1610612743   
518    1630533  2025-26  Ziaire Williams  1610612751   
519    1629627  2025-26  Zion Williamson  1610612740   

     DIST_MILES_OFF_SpeedDistance  DIST_MILES_DEF_SpeedDistance  \
0                             0.4                           0.4   
1                             1.2                           1.0   
2                             0.4                           0.3   
3                             1.0                          

In [25]:
# Define columns to keep as grouping keys (non-numeric)
group_keys = ['PLAYER_ID', 'age', 'season', 'PLAYER_NAME', 'TEAM_ID']

# Identify numeric columns to convert to percentiles
numeric_cols = [col for col in playtype_merged_df.columns if col not in group_keys]

# Create percentile dataframe
playtype_percentile_by_age_df = playtype_merged_df[group_keys].copy()

# Calculate percentiles by age
for col in numeric_cols:
    playtype_percentile_by_age_df[col + "_pct"] = playtype_merged_df.groupby("age")[col].rank(pct=True)

# Invert percentiles for turnover-related columns
for col in numeric_cols:
    if "TOV" in col.upper():
        pct_col = col + "_pct"
        playtype_percentile_by_age_df[pct_col] = 1 - playtype_percentile_by_age_df[pct_col]

print(playtype_percentile_by_age_df)


     PLAYER_ID   age   season      PLAYER_NAME     TEAM_ID  \
0      1630639  25.0  2025-26      A.J. Lawson  1610612761   
1      1631260  26.0  2025-26         AJ Green  1610612749   
2      1642358  21.0  2025-26       AJ Johnson  1610612764   
3       203932  30.0  2025-26     Aaron Gordon  1610612743   
4      1628988  29.0  2025-26    Aaron Holiday  1610612745   
..         ...   ...      ...              ...         ...   
515    1641744  23.0  2025-26        Zach Edey  1610612763   
516     203897  30.0  2025-26      Zach LaVine  1610612758   
517    1630192  25.0  2025-26       Zeke Nnaji  1610612743   
518    1630533  24.0  2025-26  Ziaire Williams  1610612751   
519    1629627  25.0  2025-26  Zion Williamson  1610612740   

     DIST_MILES_OFF_SpeedDistance_pct  DIST_MILES_DEF_SpeedDistance_pct  \
0                            0.221154                          0.250000   
1                            0.872340                          0.819149   
2                            0

In [26]:
# Define columns to keep as grouping keys (non-numeric)
group_keys = ['PLAYER_ID', 'experience' , 'season' , 'PLAYER_NAME', 'TEAM_ID']

# Identify numeric columns to convert to percentiles
numeric_cols = [col for col in playtype_merged_df.columns if col not in group_keys]

# Create percentile dataframe
playtype_percentile_by_exp_df = playtype_merged_df[group_keys].copy()

# Calculate percentiles by experience
for col in numeric_cols:
    playtype_percentile_by_exp_df[col + "_pct"] = playtype_merged_df.groupby("experience")[col].rank(pct=True)

# Invert percentiles for turnover-related columns
for col in numeric_cols:
    if "TOV" in col.upper():
        pct_col = col + "_pct"
        playtype_percentile_by_exp_df[pct_col] = 1 - playtype_percentile_by_exp_df[pct_col]


print(playtype_percentile_by_exp_df)

     PLAYER_ID  experience   season      PLAYER_NAME     TEAM_ID  \
0      1630639         3.0  2025-26      A.J. Lawson  1610612761   
1      1631260         3.0  2025-26         AJ Green  1610612749   
2      1642358         1.0  2025-26       AJ Johnson  1610612764   
3       203932        11.0  2025-26     Aaron Gordon  1610612743   
4      1628988         7.0  2025-26    Aaron Holiday  1610612745   
..         ...         ...      ...              ...         ...   
515    1641744         1.0  2025-26        Zach Edey  1610612763   
516     203897        11.0  2025-26      Zach LaVine  1610612758   
517    1630192         5.0  2025-26       Zeke Nnaji  1610612743   
518    1630533         4.0  2025-26  Ziaire Williams  1610612751   
519    1629627         6.0  2025-26  Zion Williamson  1610612740   

     DIST_MILES_OFF_SpeedDistance_pct  DIST_MILES_DEF_SpeedDistance_pct  \
0                            0.194444                          0.231481   
1                            0.85

In [27]:
# Guard, Wing, Forward definitions
guard_positions = ["G", "G-F"]
wing_positions = ["F", "F-G"]
forward_positions = ["C", "C-F", "F-C"]

guards_df = playtype_merged_df[playtype_merged_df["position"].isin(guard_positions)]
wings_df = playtype_merged_df[playtype_merged_df["position"].isin(wing_positions)]
forwards_df = playtype_merged_df[playtype_merged_df["position"].isin(forward_positions)]



In [28]:
def calculate_percentiles_by_position_group(df):
    group_keys = ['PLAYER_ID', 'position', 'season', 'PLAYER_NAME', 'TEAM_ID']
    numeric_cols = [col for col in df.columns if col not in group_keys]
    
    percentile_df = df[group_keys].copy()
    
    for col in numeric_cols:
        percentile_df[col + "_pct"] = df[col].rank(pct=True)
        
    for col in numeric_cols:
        if "TOV" in col.upper():
            pct_col = col + "_pct"
            percentile_df[pct_col] = 1 - percentile_df[pct_col]
    
    return percentile_df

In [29]:
guard_percentiles = calculate_percentiles_by_position_group(guards_df)
wing_percentiles = calculate_percentiles_by_position_group(wings_df)
forward_percentiles = calculate_percentiles_by_position_group(forwards_df)


In [30]:
def generate_supabase_sql(df, table_name):
    sql_cols = []

    for col in df.columns:
        dtype = df[col].dtype

        if col.upper() == "PLAYER_ID":
            sql_cols.append(f'"{col}" bigint PRIMARY KEY')
        elif pd.api.types.is_integer_dtype(dtype):
            sql_cols.append(f'"{col}" bigint')
        elif pd.api.types.is_float_dtype(dtype):
            sql_cols.append(f'"{col}" double precision')
        else:
            sql_cols.append(f'"{col}" text')

    sql_statement = (
        f'CREATE TABLE "{table_name}" (\n  ' +
        ",\n  ".join(sql_cols) +
        "\n);"
    )

    return sql_statement

In [33]:
sql_create = generate_supabase_sql(synergy_merged_df, "live_synergy")
print(sql_create)

sql_create = generate_supabase_sql(playtype_values_df, "live_playtype_values")
print(sql_create) 

sql_create = generate_supabase_sql(playtype_percentile_df, "live_playtype_percentiles")
print(sql_create) 

sql_create = generate_supabase_sql(playtype_percentile_by_age_df, "live_playtype_percentiles_age")
print(sql_create) 

sql_create = generate_supabase_sql(playtype_percentile_by_exp_df, "live_playtype_percentiles_exp")
print(sql_create) 

sql_create = generate_supabase_sql(guard_percentiles, "live_guard_percentiles")
print(sql_create) 

sql_create = generate_supabase_sql(wing_percentiles, "live_wing_percentiles")
print(sql_create) 

sql_create = generate_supabase_sql(forward_percentiles, "live_forward_percentiles")
print(sql_create) 

CREATE TABLE "live_synergy" (
  "SEASON_ID" text,
  "PLAYER_ID" bigint PRIMARY KEY,
  "PLAYER_NAME" text,
  "TEAM_ID" text,
  "PLAY_TYPE" text,
  "TYPE_GROUPING" text,
  "PERCENTILE" double precision,
  "POSS_PCT" double precision,
  "PPP" double precision,
  "FG_PCT" double precision,
  "FT_POSS_PCT" double precision,
  "TOV_POSS_PCT" double precision,
  "SF_POSS_PCT" double precision,
  "PLUSONE_POSS_PCT" double precision,
  "SCORE_POSS_PCT" double precision,
  "EFG_PCT" double precision,
  "POSS" double precision,
  "PTS" double precision,
  "FGM" double precision,
  "FGA" double precision,
  "FGMX" double precision,
  "SEASON" text,
  "position" text,
  "age" double precision,
  "experience" double precision,
  "PPP_pct_by_age" double precision,
  "PPP_pct_by_experience" double precision,
  "position_group" text,
  "PPP_pct_by_position_group" double precision
);
CREATE TABLE "live_playtype_values" (
  "PLAYER_ID" bigint PRIMARY KEY,
  "season" text,
  "PLAYER_NAME" text,
  "TEAM_ID

In [36]:
records = synergy_merged_df.to_dict(orient="records")
chunk_size = 1000
for i in range(0, len(records), chunk_size):
    chunk = records[i:i + chunk_size]
    try:
        supabase.table("live_synergy").insert(chunk).execute()
        print(f"Inserted chunk {i // chunk_size + 1}")
    except Exception as e:
        print(f"Error inserting chunk {i // chunk_size + 1}: {e}")

Inserted chunk 1
Inserted chunk 2
Inserted chunk 3
Inserted chunk 4
Inserted chunk 5


In [37]:
# Replace inf and -inf with NaN first
playtype_values_df_cleaned = playtype_values_df.replace([np.inf, -np.inf], np.nan)

# Replace all NaNs with 0
playtype_values_df_cleaned = playtype_values_df_cleaned.fillna(0)


records = playtype_values_df_cleaned.to_dict(orient="records")
chunk_size = 1000
for i in range(0, len(records), chunk_size):
    chunk = records[i:i + chunk_size]
    try:
        supabase.table("live_playtype_values").insert(chunk).execute()
        print(f"Inserted chunk {i // chunk_size + 1}")
    except Exception as e:
        print(f"Error inserting chunk {i // chunk_size + 1}: {e}")

Inserted chunk 1


In [38]:
# Clean playtype_percentile_df
playtype_percentile_df_cleaned = playtype_percentile_df.replace([np.inf, -np.inf], np.nan)
playtype_percentile_df_cleaned = playtype_percentile_df_cleaned.fillna(0)

records = playtype_percentile_df_cleaned.to_dict(orient="records")
chunk_size = 1000
for i in range(0, len(records), chunk_size):
    chunk = records[i:i + chunk_size]
    try:
        supabase.table("live_playtype_percentiles").insert(chunk).execute()
        print(f"Inserted chunk {i // chunk_size + 1}")
    except Exception as e:
        print(f"Error inserting chunk {i // chunk_size + 1}: {e}")

Inserted chunk 1


In [39]:
# Clean playtype_percentile_df
playtype_percentile_by_age_df_cleaned = playtype_percentile_by_age_df.replace([np.inf, -np.inf], np.nan)
playtype_percentile_by_age_df_cleaned = playtype_percentile_by_age_df_cleaned.fillna(0)

records = playtype_percentile_by_age_df_cleaned.to_dict(orient="records")
chunk_size = 1000
for i in range(0, len(records), chunk_size):
    time.sleep(5)
    chunk = records[i:i + chunk_size]
    try:
        supabase.table("live_playtype_percentiles_age").insert(chunk).execute()
        print(f"Inserted chunk {i // chunk_size + 1}")
    except Exception as e:
        print(f"Error inserting chunk {i // chunk_size + 1}: {e}")

Inserted chunk 1


In [40]:
# Clean playtype_percentile_df
playtype_percentile_by_exp_df_cleaned = playtype_percentile_by_exp_df.replace([np.inf, -np.inf], np.nan)
playtype_percentile_by_exp_df_cleaned = playtype_percentile_by_exp_df_cleaned.fillna(0)

records = playtype_percentile_by_exp_df_cleaned.to_dict(orient="records")
chunk_size = 1000
for i in range(0, len(records), chunk_size):
    time.sleep(2)
    chunk = records[i:i + chunk_size]
    try:
        supabase.table("live_playtype_percentiles_exp").insert(chunk).execute()
        print(f"Inserted chunk {i // chunk_size + 1}")
    except Exception as e:
        print(f"Error inserting chunk {i // chunk_size + 1}: {e}")

Inserted chunk 1


In [41]:
# Clean playtype_percentile_df
guard_percentiles_cleaned = guard_percentiles.replace([np.inf, -np.inf], np.nan)
guard_percentiles_cleaned = guard_percentiles_cleaned.fillna(0)

records = guard_percentiles_cleaned.to_dict(orient="records")
chunk_size = 1000
for i in range(0, len(records), chunk_size):
    time.sleep(2)
    chunk = records[i:i + chunk_size]
    try:
        supabase.table("live_guard_percentiles").insert(chunk).execute()
        print(f"Inserted chunk {i // chunk_size + 1}")
    except Exception as e:
        print(f"Error inserting chunk {i // chunk_size + 1}: {e}")

Inserted chunk 1


In [42]:
# Clean playtype_percentile_df
wing_percentiles_cleaned = wing_percentiles.replace([np.inf, -np.inf], np.nan)
wing_percentiles_cleaned = wing_percentiles_cleaned.fillna(0)

records = wing_percentiles_cleaned.to_dict(orient="records")
chunk_size = 1000
for i in range(0, len(records), chunk_size):
    time.sleep(2)
    chunk = records[i:i + chunk_size]
    try:
        supabase.table("live_wing_percentiles").insert(chunk).execute()
        print(f"Inserted chunk {i // chunk_size + 1}")
    except Exception as e:
        print(f"Error inserting chunk {i // chunk_size + 1}: {e}")

Inserted chunk 1


In [43]:
# Clean playtype_percentile_df
forward_percentiles_cleaned = forward_percentiles.replace([np.inf, -np.inf], np.nan)
forward_percentiles_cleaned = forward_percentiles_cleaned.fillna(0)

records = forward_percentiles_cleaned.to_dict(orient="records")
chunk_size = 1000
for i in range(0, len(records), chunk_size):
    time.sleep(2)
    chunk = records[i:i + chunk_size]
    try:
        supabase.table("live_forward_percentiles").insert(chunk).execute()
        print(f"Inserted chunk {i // chunk_size + 1}")
    except Exception as e:
        print(f"Error inserting chunk {i // chunk_size + 1}: {e}")

Inserted chunk 1
